In [1]:
import numpy as np
import pandas as pd

In [97]:
train1 = pd.read_csv("train.csv")
train1.head()

,userId,itemId,rating,date
0,cgexjc,682978,4,2017-08-13 13:23:35
1,cgexjc,320898,4,2019-01-18 15:56:07
2,cgexjc,29028,4,2017-08-13 14:03:55
3,cgexjc,399148,5,2017-08-13 13:59:51
4,cgexjc,734055,4,2019-01-18 15:37:29


In [98]:
test1 = pd.read_csv("test.csv")
test1.head()

,userId,itemId,date
0,cgexjc,857869,2021-08-18 00:40:19
1,cgexjc,631765,2021-08-18 01:31:23
2,cgexjc,232633,2021-08-18 01:12:46
3,cgexjc,259982,2021-08-18 00:50:49
4,cgexjc,131945,2021-08-18 00:56:13


In [9]:
df_combined = pd.merge(train1, test1, on = 'userId')
df_combined.head()

,userId,itemId_x,rating,date_x,itemId_y,date_y
0,cgexjc,682978.0,4.0,2017-08-13 13:23:35,857869,2021-08-18 00:40:19
1,cgexjc,682978.0,4.0,2017-08-13 13:23:35,631765,2021-08-18 01:31:23
2,cgexjc,682978.0,4.0,2017-08-13 13:23:35,232633,2021-08-18 01:12:46
3,cgexjc,682978.0,4.0,2017-08-13 13:23:35,259982,2021-08-18 00:50:49
4,cgexjc,682978.0,4.0,2017-08-13 13:23:35,131945,2021-08-18 00:56:13


In [10]:
df_combined.shape

(1769258, 6)

# روش اول

In [44]:
from keras.layers import Embedding, Input, dot, concatenate
from keras.models import Model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from sklearn.model_selection import train_test_split

In [28]:
X = train1.iloc[:,:2]
Y = train1.iloc[:,2]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 66)

In [29]:
n_latent_factors = 50
n_users, n_items = len(train1['userId'].unique()), len(train['itemId'].unique()) 

In [60]:
user_input = Input(shape=(1,), name='User_Input')
user_embeddings = Embedding(input_dim = n_users, output_dim=n_latent_factors, input_length=1, 
                              name='User_Embedding') (user_input)
user_vector = Flatten(name='User_Vector') (user_embeddings)



item_input = Input(shape=(1,), name='Item_Input')
item_embeddings = Embedding(input_dim = n_items, output_dim=n_latent_factors, input_length=1, 
                               name='Item_Embedding') (item_input)
item_vector = Flatten(name='Item_Vector') (item_embeddings)



merged_vectors = dot([user_vector, item_vector], name='Dot_Product', axes=1)
model = Model([user_input, item_input], merged_vectors)

In [95]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Item (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 User (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 ItemEmbedding (Embedding)      (None, 1, 50)        737700      ['Item[0][0]']                   
                                                                                                  
 UserEmbedding (Embedding)      (None, 1, 50)        66900       ['User[0][0]']                   
                                                                                            

In [61]:
optimizer = Adam(lr = 0.0005)

model.compile(loss='mean_squared_error', optimizer = optimizer)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
history = model.fit(x=[x_train['userId'], x_train['itemId']], y=y_train, batch_size=128, epochs=10, verbose= 2, validation_data=([x_test['userId'], x_test['itemId']], y_test))

# روش دوم

In [ ]:
pip install reco

In [ ]:
from reco.preprocess import user_split

In [69]:
n_users = train1.userId.nunique()
n_items = train1.itemId.nunique()

In [70]:
max_rating = train1.rating.max()
min_rating = train1.rating.min()

In [ ]:
train, test = User_split(train, [0.9, 0.1])

In [72]:
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Dot, Add, Lambda, Activation, Reshape, Concatenate, Dense, Dropout
from keras.regularizers import l2
from keras.constraints import non_neg
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot

In [73]:
def Deep_MF(n_users, n_items, n_factors):
    

    item_input = Input(shape=[1], name='ItemId')
    item_embedding = Embedding(n_items, n_factors, embeddings_regularizer=l2(1e-6),
                               embeddings_initializer='glorot_normal',
                               name='ItemEmbedding')(item_input)
    item_vec = Flatten(name='FlattenItemE')(item_embedding)
    

    item_bias = Embedding(n_items, 1, embeddings_regularizer=l2(1e-6), 
                          embeddings_initializer='glorot_normal',
                          name='ItemBias')(item_input)
    item_bias_vec = Flatten(name='FlattenItemBiasE')(item_bias)


    user_input = Input(shape=[1], name='UserId')
    user_embedding = Embedding(n_users, n_factors, embeddings_regularizer=l2(1e-6),
                               embeddings_initializer='glorot_normal',
                               name='UserEmbedding')(user_input)
    user_vec = Flatten(name='FlattenUserE')(user_embedding)


    user_bias = Embedding(n_users, 1, embeddings_regularizer=l2(1e-6), 
                        embeddings_initializer='glorot_normal',
                          name='UserBias')(user_input)
    user_bias_vec = Flatten(name='FlattenUserBiasE')(user_bias)


    Concat = Concatenate(name='Concat')([item_vec, user_vec])
    ConcatDrop = Dropout(0.5)(Concat)

    kernel_initializer='he_normal'
    

    Dense_1 = Dense(10, kernel_initializer='glorot_normal', name="Dense1")(ConcatDrop)
    Dense_1_Drop = Dropout(0.5)(Dense_1)
    Dense_2 = Dense(1, kernel_initializer='glorot_normal', name="Dense2")(Dense_1_Drop)

    
    AddBias = Add(name="AddBias")([Dense_2, item_bias_vec, user_bias_vec])
    
    
    

    y = Activation('sigmoid')(AddBias)
    rating_output = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(y)
    

    model = Model([user_input, item_input], rating_output)
    

    model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001))
    
    return model

In [74]:
n_factors = 50
model = Deep_MF(n_users, n_items, n_factors)

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [75]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Item (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 User (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 ItemEmbedding (Embedding)      (None, 1, 50)        737700      ['Item[0][0]']                   
                                                                                                  
 UserEmbedding (Embedding)      (None, 1, 50)        66900       ['User[0][0]']                   
                                                                                            

In [ ]:
output = model.fit([train.userId, train.itemId], train.rating, 
                                  batch_size=128, epochs=5, verbose=1, 
                   validation_data= ([test.userId, test.itemId], test.rating))